In [1]:
# Installing External Dependencies
# Se estiver rodando no vscode, trocar ! por %
%pip install tabulate


Note: you may need to restart the kernel to use updated packages.


In [31]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate as tab


In [32]:
# Utility Functions

def parseToNum(num_string):
    if (num_string) and ('.' in num_string) and (not '?' in num_string):
        return float(num_string)
    elif (num_string) and (not '?' in num_string):
        return int(num_string)
    elif num_string == '?':
      return -1

def is_lvdd_abnormal(lvdd):
  return 5.9 <= lvdd < 6.8

def is_epss_abnormal(lvdd):
  return lvdd > 7


In [ ]:
# Getting and parsing the data sample

data_file = open('./echocardiogram.data', 'r')
file_lines = data_file.readlines() # le as linhas e as guarda em uma lista

cleared_data = []
total_records = len(file_lines)

# Processing the data
for lineIndex in range(len(file_lines)):
    line = file_lines[lineIndex].split(',')

    line[0] = parseToNum(line[0])
    line[1] = parseToNum(line[1])
    line[5] = parseToNum(line[5])
    line[6] = parseToNum(line[6])

    cleared_data.append([line[0], line[1], line[5], line[6]])

# Creating Table
table = tab(cleared_data, 
            headers = ["Survival Time", "is Alive", "EPSS", "LVDD"], 
            showindex = "always",
            tablefmt="github",
            numalign="center")

print(table)

## **Primeira Análise**

Agora que temos os dados que serão usados agrupados em uma tabela, podemos trabalhar com eles. Primeiro, uma observação, os dados inexistentes referentes as colunas das tabelas serão representados por um **-1**. Isso irá facilitar ao excluir a linha da análise, bem como ficar fácil de visualizar a inexistência desse dado.

A nossa análise será a seguinte:


> Dado que o indivíduo sofreu um ataque cardíaco, qual a probabilidade que ele tenha níveis anormais de LVDD e EPSS

Isso nos permite ter uma noção da relação entre essas medidas e a ocorrência de ataques cardíacos em indvíduos, já que possuimos uma amostra aleatória com vários dados. Sabemos que um EPSS acima de 7mm, segundo o relatório entregue, indica que um indivíduo possui uma fação de ejeção do ventrículo esquerdo baixa, o que causa um acumulo de pressão nele, podendo gerar um ataque cardíaco.

Quanto ao LVDD, taxas altas de LVDD, no range **5.9 <= lvdd < 6.8**, indica a presença de corações doentes, segundo a própria base de dados.


In [38]:
abnormal_lvdd_amount = 0
abnormal_epss_amount = 0
abnormal_epss_and_lvdd = 0

# Looping the data list

for survival, isAlive, epss, lvdd in cleared_data:
  if is_lvdd_abnormal(lvdd) and not is_epss_abnormal(epss):
    abnormal_lvdd_amount = abnormal_lvdd_amount + 1
  elif is_epss_abnormal(epss) and not is_lvdd_abnormal(lvdd):
    abnormal_epss_amount = abnormal_epss_amount + 1
  elif is_epss_abnormal(epss) and is_lvdd_abnormal(lvdd):
    abnormal_epss_and_lvdd = abnormal_epss_and_lvdd + 1

# Para uma pessoa com apenas niveis anormais de LVDD, a chance de ataque card
print(abnormal_lvdd_amount / total_records)

# Para uma pessoa com apenas niveis anormais de EPSS, a chance de ataque card
print(abnormal_epss_amount / total_records)

# Para uma pessoa com niveis anormais de EPSS e LVDD ao mesmo tempo, a chance de ataque card
print(abnormal_epss_and_lvdd / total_records)


1 77 9
0.007575757575757576
0.5833333333333334
0.06818181818181818


A partir dos valores obtidos, vemos que a ocorrência de ataques cardíacos tem uma relação maior com níveis elevados de EPSS, chegando em torno de 58,34% a chance de alguém que sofreu um evento desse possuir taxas acima dos 7mm (o nível anormal).

## **Segunda Análise**

Uma segunda análise permite, a partir dos dados, realizar um teste de hipóteses com a proposta de, com essa amostra, provar se uma pessoa vai ou não morrer devido à um segundo ataque cardíco dentro de ano após o primeiro.